<img src="logo.png" width="200">

<b>Visão geral das contas de determinado período:</b><br><br>

In [31]:
#Instalando e importando bibliotecas
%matplotlib inline
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from IPython.display import display, HTML
#plt.rcParams['axes.formatter.use_locale'] = True
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
import squarify
from ipywidgets import interact, interactive, fixed, interact_manual
import os

In [32]:
#Instalando e importando bibliotecas
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
!jupyter nbextension enable --py widgetsnbextension
# Standard plotly imports
from chart_studio.plotly import plot, iplot as py
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [33]:
#Ajustando formato de resultados
pd.options.display.float_format = '{:,.2f}'.format

from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

#SQL PARA GERAR O ARQUIVO cp_contas_a_pagar.xlsx (EXTRAIR UM ARQUIVO PARA CADA ANO, VIA FERRAMENTA DE SQL - NO MEU CASO NÃO PODIA CONECTAR DIRETAMENTE AO BANCO):
SELECT * 
 FROM CONTAS
WHERE DATA BETWEEN '01-JAN-20XX' AND '31-DEZ-20XX' ---AJUSTAR O PERÍODO



#SQL PARA GERAR O ARQUIVO cp_favorecido.xlsx (EXTRAIR VIA FERRAMENTA DE SQL) :
SELECT 
       *
FROM fornecedor

In [34]:
#Função para escolha do ano a analisar, ajustar conforme as extrações feitas em SQL
ano_escolhido = '2020'
def show_stats_by_tag(ano):
    global ano_escolhido
    ano_escolhido = '2020'
    ano_escolhido = (f'{ano}')
    return


In [77]:
#Função para carregar a base escolhida
def carregar_base():
    global contas
    path = 'bases'
    name = '%s.xlsx' %ano_escolhido
    endereco_arquivo = os.path.join(path, name)



    #Leitura de arquivo do excel contendo histórico de contas a pagar e descrição de siglas
    siglas_favorecidos = pd.read_excel('siglas_favorecidos_pagamentos.xlsx', sheet_name='SIGLAS_FAVORECIDOS', 
                                       dtype={'SIGLA_FAVORECIDO': 'str', 'TIPO_FAVORECIDO_DESCRICAO': 'str'})
    siglas_tipo_pagamento = pd.read_excel('siglas_favorecidos_pagamentos.xlsx', sheet_name='SIGLAS_TIPO_PAGAMENTO',
                                          dtype={'SIGLA_TIPO_PAGAMENTO': 'str', 'TIPO_PAGAMENTO_DESCRICAO': 'str'})
    contas = pd.read_excel(endereco_arquivo, dtype={'SEQUENCIA': 'str', 'EMPRESA': 'str', 'DOCUMENTO': 'str',
                                                    'DATA': 'str', 'VALOR': 'float64', 'PARCELAS': 'int', 'FAVORECIDO': 'str',
                                                    'CNPJ_CPF_FAVORECIDO': 'str','TIPO_FAVORECIDO': 'str',
                                                    'TIPO_PAGAMENTO': 'str', 'STATUS': 'str', 'USUARIO_LIBERACAO': 'str',
                                                    'DATA_LIBERACAO': 'str','DATA_VENCIMENTO': 'str', 
                                                    'DATA_LIQUIDACAO': 'str', 'USUARIO_LIQUIDACAO': 'str', 
                                                    'CENTRO_CUSTO': 'str'})
    contas['DATA'] = pd.to_datetime(contas['DATA']).dt.strftime('%d/%m/%Y')
    contas['DATA_LIBERACAO'] = pd.to_datetime(contas['DATA_LIBERACAO']).dt.strftime('%d/%m/%Y')
    contas['DATA_VENCIMENTO'] = pd.to_datetime(contas['DATA_VENCIMENTO']).dt.strftime('%d/%m/%Y')
    contas['DATA_LIQUIDACAO'] = pd.to_datetime(contas['DATA_LIQUIDACAO']).dt.strftime('%d/%m/%Y')
    #contas.head(3)
    
    #Unindo as bases carregadas
    contas_com_favorecidos = pd.merge(contas,siglas_favorecidos, left_on='TIPO_FAVORECIDO', right_on='SIGLA_FAVORECIDO', 
                                      how='left')
    contas_com_tipo_pagamento = pd.merge(contas_com_favorecidos,siglas_tipo_pagamento, left_on='TIPO_PAGAMENTO', 
                                     right_on='SIGLA_TIPO_PAGAMENTO', how='left')
    contas = contas_com_tipo_pagamento
    contas=contas.drop(['SIGLA_FAVORECIDO','SIGLA_TIPO_PAGAMENTO','FAVORECIDO'], axis=1)
    del contas_com_favorecidos
    del contas_com_tipo_pagamento

    #Exibindo primeiras cinco linhas da base de contas importada do banco de dados
    #display(HTML(contas.head().to_html(index=False)))
    #Exibindo soma de valor transacionado por tipo de favorecido e tipo de pagamento
    #soma_por_tipo = contas.groupby(['TIPO_FAVORECIDO_DESCRICAO','TIPO_PAGAMENTO_DESCRICAO'])[['VALOR']].sum()
    #display(HTML(soma_por_tipo.to_html(index=True)))
    return contas

In [54]:
#Criando função pra exibir os valores de cada favorecido, 
#para um grupo de tipo de pagamento e de tipo de favorecido escolhido pelo usuário
def fun_detalhe_a():
    global detalhe_a
    detalhe_a = contas_selecionadas_usuario
    global detalhe_b
    detalhe_b = detalhe_a.groupby(['CNPJ_CPF_FAVORECIDO'],as_index=False)[['VALOR']].sum()
    detalhe_b.sort_values(by='VALOR', ascending=False)
    return (detalhe_a, detalhe_b);

In [55]:
#Criando função para obter o valor médio e o limite superior de análise com dois desvios
def fun_limite_superior():
    media = detalhe_b['VALOR'].mean()
    mediana = detalhe_b['VALOR'].median()
    desvio_padrao = detalhe_b['VALOR'].std(ddof=0)
    global limite_superior
    limite_superior = media + 2* desvio_padrao
    print('\033[1m' + '\nPara o grupo selecionado, foram calculados os valores a seguir:')
    print('\033[0m' + 'Média = ',f'{media:,.2f}')
    print('Mediana = ',f'{mediana:,.2f}')
    print('Desvio padrão = ',f'{desvio_padrao:,.2f}')
    print('Limite superior = ',f'{limite_superior:,.2f}')
    return(limite_superior);

In [56]:
#Criando função para definir quais os favorecidos que serão analisados no grupo
#trazendo dados dos favorecidos, de outra base em excel

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=DeprecationWarning)

def fun_detalhe_c_com_dados():

    import plotly.graph_objs
    detalhe_c = detalhe_b.loc[(detalhe_b['VALOR'] > limite_superior)]
    print('\033[1m' + 'Considerando o limite superior do grupo, cabe análise mais detalhada dos favorecidos a seguir:')
    
    base_favorecidos = pd.read_excel('cp_favorecido.xlsx', 
                                     dtype={'CODIGO': 'str', 'CPF_CNPJ_FAVORECIDO': 'str', 'RAZAO_SOCIAL': 'str'})
    
    base_favorecidos.CNPJ_CPF_FAVORECIDO = base_favorecidos.CNPJ_CPF_FAVORECIDO.astype('str')
    base_favorecidos = base_favorecidos.sort_values(by=['CNPJ_CPF_FAVORECIDO','CODIGO'])
    base_favorecidos = base_favorecidos.drop_duplicates(subset='CNPJ_CPF_FAVORECIDO', keep="last")
    
    
    #base_favorecidos = base_favorecidos.rename(columns={"CPF_CNPJ":"CNPJ_CPF_FAVORECIDO"})
    base_favorecidos=base_favorecidos.drop(['CODIGO'], axis=1)
    
    global detalhe_c_com_dados
    detalhe_c_com_dados = pd.merge(detalhe_c,base_favorecidos, how='left', on='CNPJ_CPF_FAVORECIDO')
    detalhe_c_com_dados['RAZAO_SOCIAL'].fillna('N localizado base_favorecidos', inplace=True)
    detalhe_c_com_dados.sort_values(by='VALOR', ascending=False)
    trace1 = go.Bar(
                x = detalhe_c_com_dados.RAZAO_SOCIAL,
                y = detalhe_c_com_dados.VALOR,
                name = "VALOR",
                marker = dict(color = '#008295',
                             line=dict(color='rgb(0,0,0)',width=1),
                             opacity=0.6))
    data = [trace1]
    layout = go.Layout(barmode = "group",xaxis = go.XAxis(title = 'Favorecido',showticklabels=False),)
    fig3 = go.Figure(data = data, layout = layout)
    #fig3.update_layout(autosize=False,width=500,height=500)
    #fig3.update_xaxes(side='right',tickangle=45,automargin=False)

    iplot(fig3) 

    if (detalhe_c_com_dados).shape[0] == 0:
        norm = matplotlib.colors.Normalize(vmin=(1), vmax=(1))
    else:
        norm = matplotlib.colors.Normalize(vmin=min(detalhe_c_com_dados.VALOR), vmax=max(detalhe_c_com_dados.VALOR))
    
    colors = [matplotlib.cm.Blues(norm(value)) for value in detalhe_c_com_dados.VALOR]
    fig = plt.gcf()
    ax = fig.add_subplot()
    ax.set_xticks([])
    ax.set_yticks([])
    fig.set_size_inches(18, 8)
    labels = ["%s\n%.2f" % (label) for label in zip(detalhe_c_com_dados.RAZAO_SOCIAL, detalhe_c_com_dados.VALOR)]
    squarify.plot(label=labels,
                  text_kwargs={'fontsize':10, 'fontname':"Arial",'weight':'bold'},
                  sizes=detalhe_c_com_dados.VALOR, color = colors, alpha=.4)
    plt.title("Valor contas por favorecidos acima do limite superior",fontsize=14,fontweight="bold")
    plt.axis('off')
    plt.gcf()
    plt.show()

    return;

In [57]:
#Criando função para gerar tabela com detalhes dos favorecidos acima do limite superior
def fun_tab_detalhe_e():
    soma_por_favorecido = detalhe_a[['CNPJ_CPF_FAVORECIDO', 'VALOR']].groupby(('CNPJ_CPF_FAVORECIDO'),as_index=False).sum()
    max_por_favorecido = detalhe_a[['CNPJ_CPF_FAVORECIDO', 'VALOR']].groupby(('CNPJ_CPF_FAVORECIDO'),as_index=False).max()
    min_por_favorecido = detalhe_a[['CNPJ_CPF_FAVORECIDO', 'VALOR']].groupby(('CNPJ_CPF_FAVORECIDO'),as_index=False).min()
    
    soma_por_favorecido = soma_por_favorecido.rename(columns={"VALOR":"SOMA_VALOR", 'CNPJ_CPF_FAVORECIDO':'CNPJ_CPF_FAVORECIDO'})
    max_por_favorecido = max_por_favorecido.rename(columns={"VALOR":"MAX_VALOR", 'CNPJ_CPF_FAVORECIDO':'CNPJ_CPF_FAVORECIDO'})
    min_por_favorecido = min_por_favorecido.rename(columns={"VALOR":"MIN_VALOR", 'CNPJ_CPF_FAVORECIDO':'CNPJ_CPF_FAVORECIDO'})
    
    max_min = pd.merge(max_por_favorecido, min_por_favorecido, on='CNPJ_CPF_FAVORECIDO', how='left')
    
    detalhe_e = pd.merge(detalhe_c_com_dados, max_min, left_on='CNPJ_CPF_FAVORECIDO', right_on='CNPJ_CPF_FAVORECIDO', how='inner')
    #detalhe_e = detalhe_e.drop(['CODIGO'], axis=1)
    
    display(HTML(detalhe_e.sort_values(by='VALOR', ascending=False).to_html(index=False)))
        
    return;

In [58]:
#Listando CNPJ_CPF_FAVORECIDO para consultas externas
def lista_cnpj_para_consultar():
    lista_cnpjs_para_analisar = pd.Series(detalhe_c_com_dados['CNPJ_CPF_FAVORECIDO'].unique()).to_list()
    #lista_cnpjs_para_analisar_2 = lista_cnpjs_para_analisar.split(",")
    return print('\nLista dos cnpjs exibidos, para consultas externas:', lista_cnpjs_para_analisar, '\n')

In [59]:
#Criando função que unifica as demais de detalhamento
def consultar():
    
    fun_detalhe_a()
    fun_limite_superior()
    fun_detalhe_c_com_dados()
    fun_tab_detalhe_e()
    lista_cnpj_para_consultar()
    return;

In [60]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

In [61]:
#Criando dropdowns
def dropdowns():
    global dropdown_tipo_favorecido,  dropdown_tipo_pagamento 
    global output_tipos, plot_output_favorecido, plot_output_pagamento, detalhamento

    contas = carregar_base()
    dropdown_tipo_favorecido = widgets.Dropdown(options = unique_sorted_values_plus_ALL(contas.TIPO_FAVORECIDO_DESCRICAO))
    dropdown_tipo_pagamento = widgets.Dropdown(options = unique_sorted_values_plus_ALL(contas.TIPO_PAGAMENTO_DESCRICAO.astype('str')))

        
    output_tipos = widgets.Output()
    plot_output_favorecido = widgets.Output()
    plot_output_pagamento = widgets.Output()
    detalhamento = widgets.Output()
    
    return;

In [62]:
#Transformando dropdowns em filtros
def common_filtering(tipo_favorecido, tipo_pagamento):
    global common_filter
    global contas_selecionadas_usuario
    
    output_tipos.clear_output()
    plot_output_favorecido.clear_output()
    plot_output_pagamento.clear_output()
    detalhamento.clear_output()
    
    if (tipo_favorecido == ALL) & (tipo_pagamento == ALL):
        common_filter = contas
    elif (tipo_favorecido == ALL):
        common_filter = contas[contas.TIPO_PAGAMENTO_DESCRICAO == tipo_pagamento]
    elif (tipo_pagamento == ALL):
        common_filter = contas[contas.TIPO_FAVORECIDO_DESCRICAO == tipo_favorecido]
    else:
        common_filter = contas[(contas.TIPO_FAVORECIDO_DESCRICAO == tipo_favorecido) & 
                                  (contas.TIPO_PAGAMENTO_DESCRICAO == tipo_pagamento)]
    contas_selecionadas_usuario = common_filter
    
    with output_tipos:
        display(common_filter)
    with plot_output_favorecido:
        fig2 = contas_selecionadas_usuario.groupby(['TIPO_FAVORECIDO_DESCRICAO'])[['VALOR']].sum().iplot(kind='bar', color='#008295', xTitle='Tipo de favorecido', yTitle='Soma', title='Valor total por tipo de favorecido')
    with plot_output_pagamento:
        fig1 = contas_selecionadas_usuario.groupby(['TIPO_PAGAMENTO_DESCRICAO'])[['VALOR']].sum().iplot(kind='bar', color='#008295', xTitle='Tipo de pagamento', yTitle='Soma', title='Valor total por tipo de pagamento')
    with detalhamento:
        consultar()

In [63]:
#Criando relação de filtros para ambos dropdowns criados
def dropdown_tipo_favorecido_eventhandler(change):
    common_filtering(change.new, dropdown_tipo_pagamento.value)
def dropdown_tipo_pagamento_eventhandler(change):
    common_filtering(dropdown_tipo_favorecido.value, change.new)

In [64]:

botao_iniciar_resultado = widgets.Output(layout={'border': '1px solid black'})

@botao_iniciar_resultado.capture(clear_output=True)
def iniciando(event):
    print('Você clicou no botão iniciar')
    dropdowns()
    resultado_dropdowns()
    layouts_widgets()
    return 1.0 

botao_iniciar = widgets.Button(
    description='Carregar base selecionada',
    layout={'width': '200px'}
)
botao_iniciar.on_click(iniciando)

##até aqui, 06s

In [65]:
def resultado_dropdowns():
    dropdown_tipo_favorecido.observe(dropdown_tipo_favorecido_eventhandler, names='value')
    dropdown_tipo_pagamento.observe(dropdown_tipo_pagamento_eventhandler, names='value')
    return;

In [66]:
def layouts_widgets():
    global item_layout, input_widgets
    item_layout = widgets.Layout(margin='0 0 50px 0')
    input_widgets = widgets.HBox([dropdown_tipo_favorecido, dropdown_tipo_pagamento], layout=item_layout)
    return;

In [67]:
def exibir_resultados():
    layouts_widgets()
    tab = widgets.Tab([output_tipos, plot_output_favorecido, plot_output_pagamento, detalhamento], layout=item_layout)
    tab.set_title(0, 'Visualização da base de dados')
    tab.set_title(1, 'Totais por favorecido')
    tab.set_title(2, 'Totais por pagamento')
    tab.set_title(3, 'Detalhamento')
    dashboard = widgets.VBox([input_widgets, tab])
    display(dashboard)
    return;

In [68]:
#Exibir o botão para escolha do ano    
def exibir_botao_ano_e_carregar():
    stats = interact(show_stats_by_tag,ano=widgets.ToggleButtons(options=['2020','2019', '2018']))
    return;

<font color='darkblue'>
<br><br><br><b>Detalhamento de contas:</b><br><br>
</font>

In [69]:
exibir_botao_ano_e_carregar()
botao_iniciar

interactive(children=(ToggleButtons(description='ano', options=('2020', '2019', '2018'), value='2020'), Output…

Button(description='Carregar base selecionada', layout=Layout(width='200px'), style=ButtonStyle())

<font color='darkblue'>
<b><br>Executar todas as células anteriores. Após concluído o carregamento da base, executar célula seguinte.<br></b><br>
</font>

In [83]:
exibir_resultados()

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>